In [ ]:
!dir /b /w .

In [ ]:
import sys
import os
import torch
sys.path.append(os.path.abspath('../python'))

cuda = torch.cuda.is_available()
dtype = torch.float
from PIL import Image
from imageProcess import toTorch, readFile, toOutput
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
show = lambda im: Image.fromarray(toOutput(8)(im).transpose(1, 2, 0))
readFile = readFile()
modelFile = 'lite4.onnx'

In [ ]:
from MoeNet_lite2 import Net
model = Net(4)
model.load_state_dict(torch.load('../model/lite/model_4.pth'))
model.eval()
for param in model.parameters():
    param.requires_grad_(False)
weights = model.state_dict()
for name in weights.keys():
    print(name)

In [ ]:
modelPath = './model_epoch_43.pth'
o = torch.load(modelPath, map_location='cpu')
for key in o:
    print(key)

In [ ]:
w = o['model'].state_dict()
for name in w.keys():
    print(name)

In [ ]:
for name in weights.keys():
    weights[name] = w['module.' + name]

In [ ]:
def changeName(old, new):
    weights[new] = weights[old]
    del weights[old]

names = ['conv_1.weight', 'conv_2.weight', 'relu.weight', 'scale.scale']
for i in range(6):
    for name in names:
        changeName('convt_F1{}.{}'.format(i + 1, name), 'convt_F{}.{}'.format(i + 1, name))
changeName('convt_shape1.weight', 'u.weight')

In [ ]:
model.load_state_dict(weights)
torch.save(model.state_dict(), '../model/lite/model_4.pth')

In [ ]:
fileName = '../images/example1s.png'
#fileName = 'example1s.png'
original = toTorch(8, dtype, device)(readFile(fileName))
if cuda:
    original = original.cuda()
x = original.unsqueeze(1)

In [ ]:
if cuda:
    model = model.cuda()

In [ ]:
out = model(x)[0].squeeze(1)

In [ ]:
from torch import nn
import torch.onnx
torch.onnx.export(model, x, modelFile, export_params=True, opset_version=10,
                  do_constant_folding=True, keep_initializers_as_inputs=True,
                  input_names=['x'], output_names=['out'],
                  dynamic_axes={'x': {0: 'channels', 2: 'height', 3: 'weight'}, 'out': {0: 'channels', 2: 'height', 3: 'weight'}})

In [ ]:
import onnx

onnx_model = onnx.load(modelFile)
onnx.checker.check_model(onnx_model)

In [ ]:
import onnxruntime

ort_session = onnxruntime.InferenceSession(modelFile)

In [ ]:
ort_inputs = {'x': x.cpu().numpy()}

In [ ]:
ort_out = ort_session.run(None, ort_inputs)[0].squeeze(1)

In [ ]:
import numpy as np
np.testing.assert_allclose(out.numpy(), ort_out, rtol=1e-03, atol=1e-05)

In [ ]:
show(torch.from_numpy(ort_out))

In [ ]:
show(out)